In [1]:
from cellpose import models
from tqdm import tqdm
import os
import tifffile
import numpy as np

python_test = __import__('0_python_test')

In [ ]:
raw_path = 'D:\\Bestun\\raw_data\\ModelTest\\'
model_path = 'D:\\Bestun\\training_images_for_cellpose\\cellpose_train\\models\\cellpose_residual_on_style_on_concatenation_off_cellpose_train_2025_02_10_09_52_21.944827'
save_path = 'D:\\Bestun\\masks_tracked\\ModelTest\\'

images = ['231224_cluster_1h_2.tif']

occurrence_limit = 80       # Percentage of images a cell has to be in order to be accounted for

'''
dist_limit:
    The longest distance, in pixels, that the center of mass is allowed
    to move from one image to the next for it to still count as the same
    cell. Make bigger if cells are moving a lot.

backtrack_limit:
    The maximum number of images back that the algorithm will search
    through to find a center of mass within the distance limit (dist_limit).
    Make smaller if cells are moving a lot.
    Can be adjusted in the get_tracked_masks function in 0_python_test.py
'''

#TODO: evaluate the effect of different settings for diameter and dist_limit 
#TODO: evaluate the effect of changing flow_threshold
#TODO: continue looking at the tracking function to make it avoid putting the same label on several cells

T = 5                       # Period between images (seconds)

''' If generating new masks '''
diam = 30                   # Standard setting when trained
dist_limit = diam / 2       # Slightly less than half average diameter
#backtrack_limit = 15

In [3]:
def track_Y(X, model, diam=diam):
    """ Optimized function to track Y using the given model. """
    # Remove empty images
    crop_idx = np.argmax(np.mean(X, axis=(1,2)) == 0)
    X = X[:crop_idx] if crop_idx > 0 else X

    # Predict labels on model
    Y = [model.eval(np.squeeze(i), diameter=diam, channels=[0,0], 
                    flow_threshold=0.4, do_3D=False)[0] for i in tqdm(np.split(X, X.shape[0]), 
                                                                     desc='Segmenting timeseries', unit=' frames')]
    
    print('\nTracking initiated...')
    return python_test.get_tracked_masks(masks=np.array(Y))

In [4]:
for image in images:
    print(f'Image name: {image}')
    if image in os.listdir(save_path):
        print(f'Segmentation for {image} already exists.')
        continue
    
    X = tifffile.imread(raw_path + image)
    
    # Load model once outside the loop if possible
    model = models.CellposeModel(gpu=True, pretrained_model=model_path)
    
    Y = track_Y(X, model)
    tifffile.imwrite(save_path + image, Y, imagej=True, metadata={'axes': 'TYX'})
    print(f'\nMasks saved.\nMasks processing complete for {image}')


Image name: 231224_cluster_1h_2.tif


FileNotFoundError: [WinError 3] The system cannot find the path specified: '\\storage3.ad.scilifelab.se\\alm\\BrismarGroup\\Bestun\\Bestun\\masks_tracked\\ModelTest\\'